In [ ]:
from bs4 import BeautifulSoup
import time
import datetime
import os
import requests
import pandas as pd
import numpy as np
import csv


In [ ]:

def new_keyword_crawling(when):

    #크롬창 안띄우고 크롤링하기
    options = webdriver.ChromeOptions()	
    options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    
    driver.implicitly_wait(5)
    driver.get('https://datalab.naver.com/keyword/realtimeList.naver?datetime=2019-04-10T'+str(when)+'%3A00%3A00')
    driver.implicitly_wait(5)

    # 날짜 및 검색 키워드 데이터를 담을 dictionary {'날짜':[검색 키워드 데이터 list]]}
    dict_datas = {}

    # 필요 없음 click_number = 0
    for click_number in range(183):
        rank_dates = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div/div[1]/a[3]/span[1]').text[:11].replace(",","")
        rank_results = driver.find_elements_by_xpath('//*[@id="content"]/div/div[3]/div/div/div[1]/div/div/ul/li/a/span')
        rank_results = rank_results[:10]

        list_rank_results = []
        for rank_result in rank_results:
            list_rank_results.append(rank_result.text)

        dict_datas[rank_dates] = list_rank_results
       
        # 날짜 변경
        date_converting = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div/div[1]/a[2]')
        date_converting.click()
    print("new_keyword_crawling 작업 완료")
    driver.close()
    return dict_datas

def old_keyword_crawling(when):
    #크롬창 안띄우고 크롤링하기
    options = webdriver.ChromeOptions()	
    options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)

    driver.implicitly_wait(5)
    driver.get('https://datalab.naver.com/keyword/realtimeList.naver?datetime=2018-04-10T'+str(when)+':00:00')
    time.sleep(3)

    # 날짜 및 검색 키워드 데이터를 담을 dictionary {'날짜':[검색 키워드 데이터 list]]}
    dict_datas = {}
    
    for click_number in range(183):
        rank_dates = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div[1]/a[3]/span[1]').text[:11].replace(",","")
        rank_results = driver.find_elements_by_xpath('//*[@id="content"]/div/div[3]/div/div[1]/div[1]/div/div/ul/li/a/span')
        rank_results = rank_results[:10]
        
        list_rank_results = []
        for rank_result in rank_results:
            list_rank_results.append(rank_result.text)

        dict_datas[rank_dates] = list_rank_results

        # 다음페이지로 넘어가기 위해 경로찾고 클릭
        date_converting = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div[1]/a[2]')
        date_converting.click()
        time.sleep(3)
    print("old_keyword_crawling 작업 완료")
    driver.close()
    return dict_datas

def dict_to_list(dict_datas):
    list_datas = []
    for dict_data in dict_datas:
        list_datas_datas = []
        list_datas_datas.append(dict_data)
        for keyword_data in dict_datas[dict_data]:
            list_datas_datas.append(keyword_data)
        list_datas.append(list_datas_datas)
    return list_datas

def new_list_to_csv(list_datas):
    dataframe_list=[]
    for list_data in list_datas:
        new_df = pd.DataFrame(np.array([list_data[1:11]]).T)
        new_df.columns = [list_data[0]]
        dataframe_list.append(new_df)
    new_df = pd.concat(dataframe_list, axis=1)
    new_df.to_csv("third_new_data/"+when+"new_list_to_csv.csv")

def old_list_to_csv(list_datas):
    dataframe_list=[]
    for list_data in list_datas:
        new_df = pd.DataFrame(np.array([list_data[1:11]]).T)
        new_df.columns = [list_data[0]]
        dataframe_list.append(new_df)
    new_df = pd.concat(dataframe_list, axis=1)
    new_df.to_csv('third_old_data/'+when+'old_list_to_csv.csv')
    
    
'''
# 8시에서 22시까지 (new)
for when in range(8, 23):
    if when <= 9:
        when = '0'+ str(when)
        new_dict = new_keyword_crawling(when)
        new_list = dict_to_list(new_dict)
        new_list_to_csv(new_list)
    else:
        when = str(when)
        new_dict = new_keyword_crawling(when)
        new_list = dict_to_list(new_dict)
        new_list_to_csv(new_list)

#8시에서 22시까지 (old) -> 쪼개서 진행
for when in range(8, 23):
    if when <= 9:
        when = '0'+ str(when)
        old_dict = old_keyword_crawling(when)
        old_list = dict_to_list(old_dict)
        old_list_to_csv(old_list)
    else:
        when = str(when)
        old_dict = old_keyword_crawling(when)
        old_list = dict_to_list(old_dict)
        old_list_to_csv(old_list)
'''